<a href="https://colab.research.google.com/github/Subhrajyoti-Dutta/QSTP/blob/main/QSTP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import the Image Data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf

In [3]:
def read_labels(path_to_labels):
    with open(path_to_labels, 'rb') as f:
        labels = np.fromfile(f, dtype=np.uint8)
        return labels

def read_images(path_to_data):
    with open(path_to_data, 'rb') as f:
        everything = np.fromfile(f, dtype=np.uint8)
        images = np.reshape(everything, (-1, 3, 96, 96))
        images = np.transpose(images, (0, 3, 2, 1))
        return images

##Retrieving training data

In [4]:
path_to_train_imgs = r"/content/drive/MyDrive/ML-AI/QSTP/train_X.bin"
path_to_train_labels = r"/content/drive/MyDrive/ML-AI/QSTP/train_y.bin"

In [5]:
Xtrain = read_images(path_to_train_imgs)
Ytrain_labels = read_labels(path_to_train_labels) - 1
Ytrain = np.zeros((5000,10),dtype=np.uint8)
Ytrain[np.arange(5000).reshape(-1,1),Ytrain_labels.reshape(-1,1)] = 1

##Retrieving test data

In [7]:
path_to_test_imgs = r"/content/drive/MyDrive/ML-AI/QSTP/test_X.bin"
path_to_test_labels = r"/content/drive/MyDrive/ML-AI/QSTP/test_y.bin"

In [8]:
Xtest = read_images(path_to_test_imgs)
Ytest_labels = read_labels(path_to_test_labels) - 1
Ytest = np.zeros((8000,10),dtype=np.uint8)
Ytest[np.arange(8000).reshape(-1,1),Ytest_labels.reshape(-1,1)] = 1

##Shuffling the data

In [10]:
Xtrain = tf.convert_to_tensor(Xtrain,dtype = tf.uint8)
Ytrain = tf.convert_to_tensor(Ytrain,dtype = tf.uint8)
Xtest  = tf.convert_to_tensor(Xtest, dtype = tf.uint8)
Ytest  = tf.convert_to_tensor(Ytest, dtype = tf.uint8)

In [11]:
indices = tf.random.shuffle(tf.range(5000))
Xtrain = tf.gather(Xtrain,indices)
Ytrain = tf.gather(Ytrain,indices)

In [12]:
indices = tf.random.shuffle(tf.range(8000))
Xtest = tf.gather(Xtest,indices)
Ytest = tf.gather(Ytest,indices)

In [13]:
print(f"Shape of Xtrain = {Xtrain.shape}")
print(f"Shape of Ytrain = {Ytrain.shape}")
print(f"Shape of Xtest  = {Xtest.shape}")
print(f"Shape of Ytest  = {Ytest.shape}")

Shape of Xtrain = (5000, 96, 96, 3)
Shape of Ytrain = (5000, 10)
Shape of Xtest  = (8000, 96, 96, 3)
Shape of Ytest  = (8000, 10)


#Save the data as .npz file for future use

Also we are saving so that we can start from middle

Uncomment the following cell to save the file midway

In [14]:
# import shutil
# np.savez("STL10-Data",Xtrain = Xtrain, Xtest = Xtest, Ytrain = Ytrain, Ytest = Ytest)
# shutil.copy("/content/STL10-Data.npz","/content/drive/MyDrive/ML-AI/QSTP/Part-1")

#Building and training the model

If the model is already trained then uncomment the following cell and start directly from here

In [15]:
# npzfile = np.load('/content/drive/MyDrive/ML-AI/QSTP/Part-1/STL10-Data.npz')
# Xtrain = npzfile['Xtrain']
# Ytrain = npzfile['Ytrain']

In [16]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64,(3,3), activation='relu', input_shape= Xtrain.shape[1:]),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Conv2D(64,(3,3), activation='relu'),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(24*24, activation = "relu"),
    tf.keras.layers.Dense(24*24, activation = 'relu'),
    tf.keras.layers.Dense(10,    activation = 'softmax'),
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 94, 94, 64)        1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 47, 47, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 45, 45, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 22, 22, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 30976)             0         
_________________________________________________________________
dense (Dense)                (None, 576)               17842752  
_________________________________________________________________
dense_1 (Dense)              (None, 576)               3

In [17]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.BinaryCrossentropy()
    )

In [18]:
model.fit(
    Xtrain,
    Ytrain,
    epochs = 10,
    callbacks = tf.keras.callbacks.EarlyStopping(
        monitor   = 'val_loss',
        patience  = 1,                                                #
        min_delta = 0.01,
        mode      = 'min'
        ),
    validation_split = 0.2,
    batch_size = 125
    )

Epoch 1/10
29/29 [==============================] - 100s 3s/step - loss: 5.7461 - val_loss: 0.3119
Epoch 2/10
29/29 [==============================] - 93s 3s/step - loss: 0.2878 - val_loss: 0.2801
Epoch 3/10
29/29 [==============================] - 94s 3s/step - loss: 0.2175 - val_loss: 0.2834


##We are saving the trained model for future use

If you want to save the model midway then uncomment the following cell

In [ ]:
# model.save("/content/drive/MyDrive/ML-AI/QSTP/Part-2")

#Testing the efficency of Model

If the model is already trained then uncomment the following two cells and start directly from here

In [20]:
# npzfile = np.load('/content/drive/MyDrive/ML-AI/QSTP/Part-1/STL10-Data.npz')
# Xtest  = npzfile['Xtest']
# Ytest  = npzfile['Ytest']

In [21]:
# model = tf.keras.models.load_model("/content/drive/MyDrive/ML-AI/QSTP/Part-2")

In [22]:
Ypred = model.predict(Xtest)                        #Predicting the y values using the trained model on Xtest

In [23]:
metric = tf.keras.metrics.BinaryAccuracy()
metric.update_state(Ytest,Ypred)
print(f"Accuracy of the model = {metric.result().numpy()}"

0.8977467